In [5]:
import numpy as np
import pandas as pd

In [7]:
# Corrected approach for binomial tree option pricing

# Define parameters
S0 = 100  # initial stock price
K = 100   # strike price (assume ATM for this case)
r = 0.05  # risk-free rate
sigma = 0.20  # volatility
T = 3 / 12  # time to maturity in years (3 months)
steps = 20  # number of steps in the binomial tree

# Calculate parameters for the binomial tree
dt = T / steps  # time increment
u = np.exp(sigma * np.sqrt(dt))  # up factor
d = 1 / u  # down factor
p = (np.exp(r * dt) - d) / (u - d)  # risk-neutral probability

# Initialize asset prices at maturity
ST = np.zeros((steps + 1, steps + 1))
for i in range(steps + 1):
    for j in range(i + 1):
        ST[j, i] = S0 * (u ** (i - j)) * (d ** j)

# Initialize option values at maturity
C = np.zeros((steps + 1, steps + 1))
P = np.zeros((steps + 1, steps + 1))
C[:, steps] = np.maximum(0, ST[:, steps] - K)  # Call option values at maturity
P[:, steps] = np.maximum(0, K - ST[:, steps])  # Put option values at maturity

# Backward induction for option prices
for i in range(steps - 1, -1, -1):
    for j in range(i + 1):
        C[j, i] = np.exp(-r * dt) * (p * C[j, i + 1] + (1 - p) * C[j + 1, i + 1])
        P[j, i] = np.exp(-r * dt) * (p * P[j, i + 1] + (1 - p) * P[j + 1, i + 1])

# Price of European Call and Put at t=0
C0 = C[0, 0]
P0 = P[0, 0]

# Calculate Delta
Delta_C = (C[0, 1] - C[1, 1]) / (ST[0, 1] - ST[1, 1])
Delta_P = (P[0, 1] - P[1, 1]) / (ST[0, 1] - ST[1, 1])

# Sensitivity to volatility (Vega)
sigma_new = 0.25  # new volatility
u_new = np.exp(sigma_new * np.sqrt(dt))
d_new = 1 / u_new
p_new = (np.exp(r * dt) - d_new) / (u_new - d_new)

# Recalculate option values at maturity with new volatility
ST_new = np.zeros((steps + 1, steps + 1))
for i in range(steps + 1):
    for j in range(i + 1):
        ST_new[j, i] = S0 * (u_new ** (i - j)) * (d_new ** j)

C_new = np.zeros((steps + 1, steps + 1))
P_new = np.zeros((steps + 1, steps + 1))
C_new[:, steps] = np.maximum(0, ST_new[:, steps] - K)
P_new[:, steps] = np.maximum(0, K - ST_new[:, steps])

for i in range(steps - 1, -1, -1):
    for j in range(i + 1):
        C_new[j, i] = np.exp(-r * dt) * (p_new * C_new[j, i + 1] + (1 - p_new) * C_new[j + 1, i + 1])
        P_new[j, i] = np.exp(-r * dt) * (p_new * P_new[j, i + 1] + (1 - p_new) * P_new[j + 1, i + 1])

# New prices with increased volatility
C0_new = C_new[0, 0]
P0_new = P_new[0, 0]

# Results
results = {
    "European Call Price (Initial Volatility)": C0,
    "European Put Price (Initial Volatility)": P0,
    "Delta (Call)": Delta_C,
    "Delta (Put)": Delta_P,
    "European Call Price (Increased Volatility)": C0_new,
    "European Put Price (Increased Volatility)": P0_new,
    "Change in Call Price due to Volatility Increase": C0_new - C0,
    "Change in Put Price due to Volatility Increase": P0_new - P0
}


results_df = pd.DataFrame([results])
results_df


,European Call Price (Initial Volatility),European Put Price (Initial Volatility),Delta (Call),Delta (Put),European Call Price (Increased Volatility),European Put Price (Increased Volatility),Change in Call Price due to Volatility Increase,Change in Put Price due to Volatility Increase
0,4.56541,3.32319,0.568612,-0.431388,5.536601,4.294381,0.971191,0.971191
